<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe30338-2021](https://jckantor.github.io/cbe30338-2021);
content is available [on Github](https://github.com/jckantor/cbe30338-2021.git).*


<!--NAVIGATION-->
< [3.2 Relay Control](https://jckantor.github.io/cbe30338-2021/03.02-Relay-Control.html) | [Contents](toc.html) | [4.0 Process Analytics](https://jckantor.github.io/cbe30338-2021/04.00-Process-Analytics.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/03.03-Lab-Assignment-PID-Control.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/03.03-Lab-Assignment-PID-Control.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 3.3 Lab Assignment 3: Relay and PI Control

This lab assignment introduces the use of relay, proportional (P), and proportional-integral (PI) control for the Temperature Control Laboratory. In this assignment you will 

1. implement and test a relay control for the dual heater/sensor system
2. implement and test a proportional-integral (PI) control algorithm for a single heater/sensor

## 3.3.1 Part 1. Relay (or On-Off) control


### 3.3.1.1 Simple relay control

The basic form relay control is described by an algorithm that sets the manipulated variable (MV) based on current values of the process variable (PV) and setpoint (SP)

\begin{align}
MV(t) & = \begin{cases} 
    MV^{max} & \text{if $PV \leq SP$}\\
    MV^{min} & \text{if $PV \geq SP$}
    \end{cases}
\end{align}

This form of relay control assumes a positive relationship between a change in MV and a corrsponding reponse in teh process variable PV. If that relationship is negative, then the logic of the controller should be inverted.

This is simple to implement and can be done with just a few lines of Python code.

### 3.3.1.2 Relay control with deadband (hysteresis)

One of the issues with simple relay control is the potential for 'chattering', which are situations where the manipulated variable leads to rapid on-and-off switching of the manipulated variable. This can be caused by systems that are highly response to control inputs, or where the sensor measurements carry significant noise.

The typical home thermostat used for furnace control incorporates a simple but highly effective solution to the chattering period. The idea is to not switch the manipulated variable off until the process variable has exceeded the setpoint by some margin, and to not switch the manipulated variable on until the process variable has fallen below the setpoint by some margin. Between these values there is no change in the manipulated variable.

Given a value $MV_k$, the next value of the manipulated variable is determined by

\begin{align}
MV(t) & = \begin{cases} 
    MV^{max} & \text{if $PV \leq SP - d$}\\
    MV^{min} & \text{if $PV \geq SP + d$}\\
    MV_k & \text{ otherwise}
    \end{cases}
\end{align}

where $d$ is the *tolerance* or *hysteresis*. 

For home heating systems a typical value is in the range of 0.5 to 1 degree F. This image shows how hystersis was adjusted on a typical home thermostat in common usage in the late 20th century.

<a title="By Vincent de Groot - http://www.videgro.net (Own work) [GFDL (http://www.gnu.org/copyleft/fdl.html), CC-BY-SA-3.0 (http://creativecommons.org/licenses/by-sa/3.0/) or CC BY 2.5 (http://creativecommons.org/licenses/by/2.5)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3AHoneywell_thermostat_open.jpg"><img width="512" alt="Honeywell thermostat open" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Honeywell_thermostat_open.jpg/512px-Honeywell_thermostat_open.jpg"/></a>

### 3.3.1.3 Exercise 1

Create a notebook to implement a relay control for the Temperature Control Lab subject the following requirements:

* Simultaneous control of sensor temperatures T1 and T2.
* Use a tolerance value $d$ of 0.5 deg C.
* Set the minimum and maximum values of the heater to 0 and 100%, respectively. lab.P1 and lab.P2 should be left at their default values.
* Show the results of an experiment in which the setpoints are adjusted as follows:
    * SP1 is initially ambient temperature. SP1 increases to 35 deg at time t=20 and remains constant.
    * SP2 is initially ambient temperature. SP2 increases linearly to 45 deg C beginning at t=120 with the transition complete at time t=220.
    * Run the experiment until you reach a steady oscillation about the final operating point, and for no shorter than  600 sec.
    
    
Questions:

   1. After the system has settled to a steady oscillation about the final operating point, what is the maximum deviation from the setpoints?
   2. What is the approximate switching frequency of the manipulated variables?

In [5]:
%matplotlib inline
from tclab import TCLab, clock, Historian, Plotter

# insert code below

## 3.3.2 Part 2. Implementing a PID Controller


## 3.3.3 PI Control

Given a process variable $PV$ and setpoint $SP$, proportional-integral control determines the value of a manipulated variable MV by the rule

\begin{align}
MV_k & = \bar{MV} + K_p(SP_k  - PV_k) + K_i\sum_{k'=0}^{k'=k-1} (SP_{k'} -  PV_{k'})
\end{align}

where $K_p$ and $K_i$, and $K_d$ are the proportional and integral gains, respectively.

### 3.3.3.1 Implementation of a simple PI controller

The actual implementation of PI control is normally done by computing how much the $MV$ should change at each time step. Defining the error at time $k$ as

\begin{align}
e_k & = SP_k - PV_k
\end{align}

then consecutive values of $MV$ are given by 

\begin{align}
MV_{k-1} & = \bar{MV} + k_p e_{k-1} + k_i \sum_{j=0}^{k-1} e_{j} \\
MV_{k} & = \bar{MV} + k_p e_{k} + k_i \sum_{j=0}^{k} e_{j}
\end{align}

Taking differences gives a practical formula for updating the value of $MV$ in response to measurements

\begin{align}
MV_{k} & = MV_{k-1} + k_p(e_{k} - e_{k-1}) + k_i e_{k}
\end{align}

### 3.3.3.2 Exercise 2.

Create a notebook to implement PI for the Temperature Control Lab.

* Using the simulation mode, create an implementaton of PI control and find values for Kp and Ki.

* Show the results of an experiment in which the setpoints are adjusted as follows:
    * SP1 is initially ambient temperature. S12 increases linearly to 45 deg C beginning at t=120 with the transition complete at time t=220.
    * Run the experiment until you reach a steady oscillation about the final operating point, and for no shorter than  600 sec.
    
    
Questions:

   1. After the system has settled to a steady oscillation about the final operating point, what is the maximum deviation from the setpoints?


The following cell provides an initial implementation of PID control for heater T1.  Modify this code to add a second controller, `pid2`, for heater T2. Test using the off-line simulator.  When happy with the results, apply the control to the actual heater.

In [1]:
from tclab import setup, clock, Historian, Plotter

TCLab = setup(connected=False, speedup = 20)

pid1 = PI()
pid1.Kp = 2
pid1.Ki = .1

with TCLab() as lab:
    h = Historian(lab.sources)
    p = Plotter(h, 800)
    for t in clock(800):
        lab.U1 = pi.update(50, lab.T1)
        p.update(t)
        

NameError: name 'PI' is not defined

<!--NAVIGATION-->
< [3.2 Relay Control](https://jckantor.github.io/cbe30338-2021/03.02-Relay-Control.html) | [Contents](toc.html) | [4.0 Process Analytics](https://jckantor.github.io/cbe30338-2021/04.00-Process-Analytics.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/03.03-Lab-Assignment-PID-Control.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/03.03-Lab-Assignment-PID-Control.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>